# Capstone Project - The Battle of Neighborhoods

# Introduction/Business Problem

I grew up in Taipei, Taiwan and Taipei is the biggest city in Taiwan in terms of population which is about 2.6 million. Taipei is a city that attract lots of tourists annually, and has many attractions for people visiting Taiwan such as one the tallest building Taipei 101. I had the opportunity to study in the United States and San Diego is one of my favorite city in US. The nick name of San Diego is the finest city in the US, attracting hundreds of thousand people every year. I am wondering if a person is debating which cities he should visit and spend one week there, which cities is his final choice.

If a travel agent would like to explore a new market, they city may consider this city’s public transportation, attractions, and room and board. Therefore, I will analyze and compare above mentioned factors and provide a recommendation for this travel agent.

# Data section

I will convert Taipei and San Diego into their equivalent latitude and longitude values, and will use the Foursquare API to explore neighborhoods in both cities. I will use the "explore" function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. I will use the *k*-means clustering algorithm to complete this task. Finally, I will use the Folium library to visualize the neighborhoods in both cities and their emerging clusters. 




In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [2]:
CLIENT_ID = 'FABS5JF1MXSOM4542ZBDJUDHXTMDNFRJAMVAJYTJ3XCJJMHF' # your Foursquare ID
CLIENT_SECRET = 'E1D400E2YXXL1E2JN3K50XCWE52XUDXPXFVHWMG11G2EWFVN' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FABS5JF1MXSOM4542ZBDJUDHXTMDNFRJAMVAJYTJ3XCJJMHF
CLIENT_SECRET:E1D400E2YXXL1E2JN3K50XCWE52XUDXPXFVHWMG11G2EWFVN


In [3]:
#address = '102 North End Ave, New York, NY'
# 2920 Zoo Dr, San Diego, CA 92101
# 7 Xinyi Road, Taipei City, Taiwan 

# address = '2920 Zoo Dr, San Diego, CA '
address = '7 Xinyi Road, Taipei City, Taiwan'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

25.1249843 121.5265369


In [4]:
#search_query = 'Italian'
search_query = 'Shopping'
radius = 35000
print(search_query + ' .... OK!')

Shopping .... OK!


In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FABS5JF1MXSOM4542ZBDJUDHXTMDNFRJAMVAJYTJ3XCJJMHF&client_secret=E1D400E2YXXL1E2JN3K50XCWE52XUDXPXFVHWMG11G2EWFVN&ll=25.1249843,121.5265369&v=20180604&query=Shopping&radius=35000&limit=30'

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5de599bc949393001c0cc397'},
 'response': {'venues': [{'id': '4bbf0882ba9776b03273fec8',
    'name': 'Miramar Shopping Mall (美麗華百樂園)',
    'location': {'address': '敬業三路20號',
     'lat': 25.083458597625867,
     'lng': 121.55658195323738,
     'labeledLatLngs': [{'label': 'display',
       'lat': 25.083458597625867,
       'lng': 121.55658195323738}],
     'distance': 5526,
     'postalCode': '104',
     'cc': 'TW',
     'neighborhood': '中山區',
     'city': '中山區',
     'state': '臺北市',
     'country': '臺灣',
     'formattedAddress': ['敬業三路20號', '中山區, 臺北市 104', '臺灣']},
    'categories': [{'id': '4bf58dd8d48988d1fd941735',
      'name': 'Shopping Mall',
      'pluralName': 'Shopping Malls',
      'shortName': 'Mall',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/mall_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1575328187',
    'hasPerk': False},
   {'id': '4cbec93fb6c4224bd803ef94',
    'name': 'Globa

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4bbf0882ba9776b03273fec8,敬業三路20號,TW,中山區,臺灣,NaN,5526,"[敬業三路20號, 中山區, 臺北市 104, 臺灣]","[{'label': 'display', 'lat': 25.08345859762586...",25.083459,121.556582,中山區,104,臺北市,Miramar Shopping Mall (美麗華百樂園),v-1575328187
1,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4cbec93fb6c4224bd803ef94,板橋區縣民大道二段7號,TW,臺灣省,臺灣,NaN,13857,"[板橋區縣民大道二段7號, 臺灣省, 臺灣省 220, 臺灣]","[{'label': 'display', 'lat': 25.01414102259211...",25.014141,121.463980,NaN,220,臺灣省,Global Shopping Center (環球購物中心 Global Shopping...,v-1575328187
2,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,565d6d6e498edbd99b0f4cdd,復興一路8號,TW,龜山區,臺灣,NaN,17380,"[復興一路8號, 龜山區, 臺灣]","[{'label': 'display', 'lat': 25.06038939995063...",25.060389,121.369576,NaN,NaN,NaN,Global Shopping Center (環球購物中心),v-1575328187
3,[],False,4ed739dd0e6145a39092c705,NaN,TW,NaN,臺灣,NaN,10609,[臺灣],"[{'label': 'display', 'lat': 25.031824, 'lng':...",25.031824,121.548725,NaN,NaN,NaN,Shopping Design設計採取買誌,v-1575328187
4,"[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",False,4b5c5399f964a520162b29e3,天母廣場,TW,台北市,臺灣,NaN,801,"[天母廣場, 台北市, 臺灣]","[{'label': 'display', 'lat': 25.11850142250345...",25.118501,121.530006,NaN,NaN,NaN,天母廣場 Tianmu Square,v-1575328187


In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Miramar Shopping Mall (美麗華百樂園),Shopping Mall,敬業三路20號,TW,中山區,臺灣,NaN,5526,"[敬業三路20號, 中山區, 臺北市 104, 臺灣]","[{'label': 'display', 'lat': 25.08345859762586...",25.083459,121.556582,中山區,104,臺北市,4bbf0882ba9776b03273fec8
1,Global Shopping Center (環球購物中心 Global Shopping...,Shopping Mall,板橋區縣民大道二段7號,TW,臺灣省,臺灣,NaN,13857,"[板橋區縣民大道二段7號, 臺灣省, 臺灣省 220, 臺灣]","[{'label': 'display', 'lat': 25.01414102259211...",25.014141,121.463980,NaN,220,臺灣省,4cbec93fb6c4224bd803ef94
2,Global Shopping Center (環球購物中心),Shopping Mall,復興一路8號,TW,龜山區,臺灣,NaN,17380,"[復興一路8號, 龜山區, 臺灣]","[{'label': 'display', 'lat': 25.06038939995063...",25.060389,121.369576,NaN,NaN,NaN,565d6d6e498edbd99b0f4cdd
3,Shopping Design設計採取買誌,None,NaN,TW,NaN,臺灣,NaN,10609,[臺灣],"[{'label': 'display', 'lat': 25.031824, 'lng':...",25.031824,121.548725,NaN,NaN,NaN,4ed739dd0e6145a39092c705
4,天母廣場 Tianmu Square,Plaza,天母廣場,TW,台北市,臺灣,NaN,801,"[天母廣場, 台北市, 臺灣]","[{'label': 'display', 'lat': 25.11850142250345...",25.118501,121.530006,NaN,NaN,NaN,4b5c5399f964a520162b29e3
5,Formosan Shopping Arcade,Shopping Mall,NaN,TW,Da-an District,臺灣,NaN,9883,"[Da-an District, 106, 臺灣]","[{'label': 'display', 'lat': 25.037471, 'lng':...",25.037471,121.543046,NaN,106,NaN,5c31b901419a9e002c2c05da
6,Taipei 101 Mall (台北101購物中心),Shopping Mall,市府路45號,TW,台北市,臺灣,NaN,10802,"[市府路45號, 台北市, 臺灣]","[{'label': 'display', 'lat': 25.03418587324281...",25.034186,121.564343,NaN,NaN,NaN,529c5d4711d2c50e964961ab
7,Pengpai Shopping Mall,Shopping Mall,NaN,TW,Zhōngshān Qū,臺灣,NaN,7896,"[中山区, 臺灣]","[{'label': 'display', 'lat': 25.05404795300935...",25.054048,121.527347,NaN,NaN,NaN,4e7adb79d22d711be729ff40
8,Shopping da Informática,Electronics Store,CLN 207,TW,NaN,臺灣,NaN,14132,"[CLN 207, 臺灣]","[{'label': 'display', 'lat': 25.176927, 'lng':...",25.176927,121.654518,NaN,NaN,NaN,4efb34ee7ee5bf7995c27e5d
9,Exquisite Shopping Center,None,NaN,TW,NaN,臺灣,NaN,7620,[臺灣],"[{'label': 'display', 'lat': 25.05844, 'lng': ...",25.058440,121.544270,NaN,NaN,NaN,51bb304d498e4cfd4ed2cb48


In [9]:
dataframe_filtered.name

0                        Miramar Shopping Mall (美麗華百樂園)
1     Global Shopping Center (環球購物中心 Global Shopping...
2                       Global Shopping Center (環球購物中心)
3                                 Shopping Design設計採取買誌
4                                    天母廣場 Tianmu Square
5                              Formosan Shopping Arcade
6                           Taipei 101 Mall (台北101購物中心)
7                                 Pengpai Shopping Mall
8                               Shopping da Informática
9                             Exquisite Shopping Center
10                                           Shopping99
11                       Zhongxiao Dunhua Shopping Area
12       OldTown White Coffee @ 1Shamelin Shopping Mall
13                              Katherine Shopping Mall
14                                             Citylink
15                   Zhongping Shopping District (中平商圈)
16                             瓦城泰國料理 Thai Town Cuisine
17                                 Global Mall (

In [14]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map